In [5]:
import pandas as pd

# Specify the file path or URL
file_path = 'AHS_KB/AHS95CallsFullDesc.xlsx'

# Read the Excel file into a DataFrame
call_df = pd.read_excel(file_path, engine='openpyxl')

In [6]:
call_df.head(n=5)

,Id,CustomerInstance,CallDate,Location,CallDuration,Transcript_with_roles,IsInbound,ProgramCode,ExtCallId,AgentName,...,SentimentClient,AgentWorkAtHome,WordCount,WordCountClient,WordCountAgent,CIProbability,APScore,APClientScore,PCAScore,Topics
0,91234836,AHS,2023-10-21 15:21:06,Davao 02,679.68,Agent: hi thank you for calling american home ...,True,AHC.USCC.CS.IBCSDD,2713075036,ronie.celiz,...,-0.026,NaN,1755,666,1089,3,24.447,34.276,0,Account_Verification (phonemes)|Cancellation (...
1,92043160,AHS,2023-10-23 12:00:08,Fairview,1611.80,Client: hello\nAgent: hi good afternoon uh goo...,False,AHC.USCC.CS.IBCSDD,2713674088,marryann.juanillo,...,-0.998,1.0,4965,3258,1707,10,23.477,17.703,0,Account_Verification (phonemes)|Cancellation (...
2,93232784,AHS,2023-10-25 18:13:18,Dasmarinas 02,749.78,Agent: \nhi good day this is ann from american...,True,AHC.USCC.CS.IBCSDD,2715035550,jennylyn.cerro,...,0.396,NaN,1805,683,1122,4,26.261,34.165,0,Account_Verification (phonemes)|Callback (phon...
3,91985634,AHS,2023-10-23 10:04:18,Dasmarinas 02,1037.34,Client: <unk> that's h. vac\nAgent: um hi good...,False,AHC.USCC.CS.IBCSDD,2713616978,jeanneiara.vedar2,...,-0.762,0.0,2957,1187,1770,20,27.951,25.951,0,Account_Verification (phonemes)|Cancellation (...
4,93234481,AHS,2023-10-25 18:32:08,Fairview,1078.52,Agent: hi this is the problem escalation may k...,False,AHC.USCC.CS.IBCSDD,2715037225,gilbert.afable,...,-0.014,NaN,2596,1454,1142,4,24.722,23.372,0,Account_Verification (phonemes)|Callback (phon...


In [33]:
# call_df["Transcript_with_roles"][0]

In [28]:
query_string = (
    "Call ID: " + str(call_df["Id"][0])
    + "\n" + "CallDate: " + str(call_df["CallDate"][0])
    + "\n" + "Location: " + str(call_df["Location"][0])
    + "\n" + "CallDuration: " + str(call_df["CallDuration"][0]) + " sec."
    + "\n" + "Transcript_with_roles: \n" + str(call_df["Transcript_with_roles"][0])
    + "\n\n" + "Summarize the call"
)

In [34]:
# query_string

In [30]:
from gradio_client import Client
import ast
HOST_URL = "http://172.20.11.71:7860"
client = Client(HOST_URL)

Loaded as API: http://172.20.11.71:7860/ ✔


In [31]:
%%time
kwargs = dict(instruction_nochat=query_string, 
        langchain_mode='LLM', 
        stream_output=False,
        max_time=120, 
        langchain_action = "Query",
        max_new_tokens=1024, 
        do_sample=False,
        document_subset='Relevant',
        document_choice='All',
        top_k_docs=4)
res = client.predict(str(dict(kwargs)), api_name='/submit_nochat_api')

CPU times: total: 15.6 ms
Wall time: 8.39 s


In [32]:
response = ast.literal_eval(res)['response']
print(response)

The caller, Linda, called American Home Shield to inquire about the status of her air conditioning repair. The agent, Ryan, confirmed that a technician had submitted a report on October 13rd but the authorization department was still pending due to lack of information. Linda explained that she had been without air conditioning for three weeks and wanted to know what the problem was and when it would be fixed. Ryan apologized and explained that their department was waiting for the technician to provide more information before making a decision. Linda asked for a call back and provided her phone number. Ryan asked her to hold while he followed up with the technician and provided documentation regarding the work order.


In [56]:
call_df["summary"] = ""

In [57]:
%%time

import time

for i in range(20):
    print (i)
    query_string = (
    "Call ID: " + str(call_df["Id"][i])
    + "\n" + "CallDate: " + str(call_df["CallDate"][i])
    + "\n" + "Location: " + str(call_df["Location"][i])
    + "\n" + "CallDuration: " + str(call_df["CallDuration"][i]) + " sec."
    + "\n" + "Transcript_with_roles: \n" + str(call_df["Transcript_with_roles"][i])
    + "\n\n" + "Summarize the call")
    
    kwargs = dict(instruction_nochat=query_string, 
        langchain_mode='LLM', 
        stream_output=False,
        max_time=120, 
        langchain_action = "Query",
        max_new_tokens=1024, 
        do_sample=False,
        document_subset='Relevant',
        document_choice='All',
        top_k_docs=4)
    
    start_time = time.time()
    res = client.predict(str(dict(kwargs)), api_name='/submit_nochat_api')
    print("--- %s seconds ---" % (time.time() - start_time))
    
    response = ast.literal_eval(res)['response']
    print(response)
    
    call_df["summary"][i] = response

0
--- 9.047047853469849 seconds ---
The caller, Linda, is frustrated with the lack of communication and updates from American Home Shield regarding her air conditioning issue. She has been waiting for over a week for a response and wants to know the status of her work order. The agent, Ron, tries to provide her with information but is unable to email her the diagnosis as his resources are limited. He offers to follow up with the technician and provide her with more information once the work order is fully authorized by the authorization department. Linda is unhappy with the service and requests to speak with a supervisor. Ron apologizes and tells her that he will put the call on hold while he tries to get in touch with the contractor to follow up on the work order.
1


<timed exec>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


--- 16.83613872528076 seconds ---
The caller is unhappy with the service provided by American Home Shield and the representative on the phone. They had a contractor come to their house and break their refrigerator, and the representative is unwilling to help them with a replacement. The caller has been paying for three years and has never used the warranty, and they are demanding to speak to someone higher up in the company. The representative keeps interrupting and not listening to the caller, and the caller becomes increasingly frustrated and angry. The representative offers to transfer the call to someone higher up, but the caller refuses and demands to cancel their service and remove their credit card from their account. The representative apologizes and agrees to cancel the service, but the caller continues to be upset and demands to speak to someone in the United States. The representative tells the caller that they cannot transfer the call and offers to process the cancellation.

In [162]:
## %%time 
query_string =""

for i in range(15):
    print (i)
    query_string += (
    "Call ID: " + str(call_df["Id"][i])
    + "\n" + "CallDate: " + str(call_df["CallDate"][i])
    + "\n" + "Location: " + str(call_df["Location"][i])
    + "\n" + "CallDuration: " + str(call_df["CallDuration"][i]) + " sec."
    + "\n" + "summary: \n" + str(call_df["summary"][i])
    +"\n\n"
    )

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [163]:
query_string += "\n\n" + "Summarize all calls by Location in details. What problems are metioned?"

In [164]:
len(query_string)

13450

In [165]:
len(query_string.split())

2230

In [166]:
#query_string

In [167]:
%%time
kwargs = dict(instruction_nochat=query_string, 
        langchain_mode='LLM', 
        stream_output=False,
        max_time=120, 
        langchain_action = "Query",
        max_new_tokens=2048, 
        do_sample=False,
        document_subset='Relevant',
        document_choice='All',
        top_k_docs=10)
res = client.predict(str(dict(kwargs)), api_name='/submit_nochat_api')

CPU times: total: 31.2 ms
Wall time: 14.9 s


In [168]:
response = ast.literal_eval(res)['response']
print(response)


Davao 02 - Caller is frustrated with the lack of communication and updates from American Home Shield regarding their air conditioning issue.
Fairview - Callers are unhappy with the service provided by American Home Shield and the representative on the phone.
Dasmarinas 02 - Callers are requesting a second opinion on the diagnosis of the technician and are unhappy with the cost of the service.
Clark 05 - Caller is trying to cancel her American Home Shield contract due to issues with their service.

Problems mentioned in the calls include:

* Lack of communication and updates from American Home Shield
* Poor service provided by the representative on the phone
* Technician not providing accurate diagnosis
* High cost of service
* Denial of service request
* Technician not covering all appliances
* Long wait for repairs
* Inconsistent communication from the company
* Unhappy with the resolution provided
* Requesting a second opinion on the diagnosis
* Canceling the contract due to poor se